# **Import Dependancies**

In [37]:
import pandas as pd
import numpy as np

import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
ps = PorterStemmer()

import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
df = pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [49]:
df = df[['text', 'airline_sentiment']]
df

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
...,...,...
14635,@AmericanAir thank you we got on a different f...,positive
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative
14637,@AmericanAir Please bring American Airlines to...,neutral
14638,"@AmericanAir you have my money, you change my ...",negative


In [52]:
df = df.drop_duplicates(keep='first')

In [53]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9087   9087   
neutral            3067   3067   
positive           2298   2298   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @VirginAmerica it's really aggressive to blast...    1  
neutral                          @VirginAmerica What @dhepburn said.    1  
positive           @VirginAmerica plus you've added commercials t...    1

In [41]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)

    # Tokenize the text
    text = nltk.word_tokenize(text)

    # Initialize an empty list to store the cleaned words
    y = []

    # Remove stopwords
    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)

    # Stem the words
    ps = PorterStemmer()
    text = y[:]  # Copy y to text
    y.clear()    # Clear y for reuse

    for i in text:
        y.append(ps.stem(i))

    # Join the cleaned words back into a single string
    return " ".join(y)

In [42]:
df['clean_text'] = df['text'].apply(clean_text)

In [43]:
df.shape

(14640, 3)

In [44]:
df.head(1)

,airline_sentiment,text,clean_text
0,neutral,@VirginAmerica What @dhepburn said.,@ virginamerica @ dhepburn said .


In [13]:
tf_vec = TfidfVectorizer(max_features=3000)
X = tf_vec.fit_transform(df['clean_text']).toarray()

In [27]:
X.shape

(14640, 3000)

In [19]:
Y = df['airline_sentiment'].values

In [26]:
Y.shape

(14640,)

In [21]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [22]:
# Create a Multinomial Naïve Bayes classifier
nb_model = MultinomialNB()

# Train the model on the training dataset
nb_model.fit(X_train, y_train)

# Predict the sentiment of the test dataset
y_pred_nb = nb_model.predict(X_test)


In [23]:
# Calculate and print the accuracy of the model
nb_accuracy = accuracy_score(y_test, y_pred_nb)
print("Naïve Bayes Model Accuracy:", nb_accuracy)

Naïve Bayes Model Accuracy: 0.7219945355191257


In [25]:
# Create a Random Forest classifier
rf_model = RandomForestClassifier()

# Train the model on the training dataset
rf_model.fit(X_train, y_train)

# Predict the sentiment of the test dataset
y_pred_rf = rf_model.predict(X_test)

# Calculate and print the accuracy of the model
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print("Random Forest Model Accuracy:", rf_accuracy)

Random Forest Model Accuracy: 0.7479508196721312
